In [67]:
import pandas as pd
import numpy as np
import re
import string

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# EDA of Full Training Dataset
Adapted from `combined-dataset-full-finetune.ipynb`

In [68]:
dat = pd.read_csv('output\\combined_trainingdata_20181013.tsv', sep='\t', low_memory=False)
test.shape

(1278129, 10)

In [69]:
dat.sample(5)
# test.head()

,index,COMPLAINT_ID,CITY,COMPLAINT DATE,DEPT_311,CODE_311,CATEGORY_MAIN,CATEGORY_SUB,COMPLAINT_1,COMPLAINT_2
1018167,104740,SEECLICKFIX_190104,National City,2018-07-05T11:36:32-04:00,NaN,abandoned vehicle,environment,environment_abandoned_vehicle,Abandoned Vehicle,junk car left for months
439396,439396,US_CINCINNATI_SR18036422,US_CINCINNATI,04/22/2018,public services,"metal furniture, spec collectn",environment,environment_garbage_collection,"""Request entered through the Web. Refer to Int...",NaN
939746,26319,SEECLICKFIX_91418,City of Albany,2018-09-30T11:30:15-04:00,NaN,tree issues,environment,environment_overgrowth,Tree Issues,The half dead tree (planted by DGS) is located...
148853,148853,US_BATON_ROUGE_227236,US_BATON_ROUGE,04/26/2016 01:24:00 PM,road maintenance issues,pothole,street,street_repair,TARA SUBDIVISION POTHOLES 6 OF 16,NaN
530550,530550,US_CINCINNATI_SR16015873,US_CINCINNATI,03/08/2016,public services,"pothole, repair",street,street_repair,"""Request entered through the Web. Refer to Int...",NaN


In [70]:
# Clean col names
dat.columns = [x.lower() for x in dat.columns]
dat.columns = [x.replace(' ', '_') for x in dat.columns]
dat.columns

Index(['index', 'complaint_id', 'city', 'complaint_date', 'dept_311',
       'code_311', 'category_main', 'category_sub', 'complaint_1',
       'complaint_2'],
      dtype='object')

In [71]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1278129 entries, 0 to 1278128
Data columns (total 10 columns):
index             1278129 non-null int64
complaint_id      1278129 non-null object
city              1278129 non-null object
complaint_date    1278127 non-null object
dept_311          913427 non-null object
code_311          1278129 non-null object
category_main     1278129 non-null object
category_sub      1278129 non-null object
complaint_1       1264104 non-null object
complaint_2       313838 non-null object
dtypes: int64(1), object(9)
memory usage: 97.5+ MB


In [72]:
dat.describe(include='all')

,index,complaint_id,city,complaint_date,dept_311,code_311,category_main,category_sub,complaint_1,complaint_2
count,1.278129e+06,1278129,1278129,1278127,913427,1278129,1278129,1278129,1264104,313838
unique,NaN,1276903,693,578885,76,1840,11,64,445976,212892
top,NaN,US_CHICAGO_10863,US_CINCINNATI,01/01/2018,public services,"metal furniture, spec collectn",environment,environment_garbage_collection,"""Request entered through the Web. Refer to Int...",MORE INFORMATION MAY BE AVAILABLE IN THE CDPH ...
freq,NaN,18,518774,23135,325608,159016,715995,400085,141283,40506
mean,3.784263e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,2.611561e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.597660e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,3.195320e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,5.938940e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Clean Data
Remove rows with no complaint 1 data. Create Complaint column that has (complaint 2 or complaint 1). We are also doing pre-processing. This involves removing punctuation, making everything lowercase, replacing numbers with N, cutting everything after 512 chars, concatenating two types of complaints when they are for Chicago.

In [73]:
# # Keep records with at least one complaint field
# dat = dat.loc[(~dat['complaint_1'].isna())|(~dat['complaint_2'].isna())]

In [74]:
dat = dat.loc[~dat['complaint_1'].isna()]

In [75]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1264104 entries, 0 to 1278128
Data columns (total 10 columns):
index             1264104 non-null int64
complaint_id      1264104 non-null object
city              1264104 non-null object
complaint_date    1264104 non-null object
dept_311          899405 non-null object
code_311          1264104 non-null object
category_main     1264104 non-null object
category_sub      1264104 non-null object
complaint_1       1264104 non-null object
complaint_2       313392 non-null object
dtypes: int64(1), object(9)
memory usage: 106.1+ MB


In [76]:
translator = str.maketrans('', '', string.punctuation) # To remove punctuation

def preProcess(complaintStart):
    complaint = complaintStart[:512] # cut to 512 characters max
    complaint = re.sub("\d","N", complaint) # remove numbers
    complaint = complaint.lower().translate(translator) # lower case and remove the punctuation
    complaint = complaint.replace("\n"," ").strip() # remove starting and trailing white spaces
    if re.search('[a-zA-Z]', complaint) is None:# if there are no letters in the complaint, return empty, will be removed in later processing
        return ""
    return complaint

def getComplaint(row):
    complaint2 = row.get("complaint_2")
    if not pd.isnull(complaint2):
        if "[INSPECTION LOG #:" in complaint2: # Remove inspection log section from C2
            complaintStrippedList = complaint2.split("]")[1:]
            complaintFinal = "]".join(complaintStrippedList)
        else:
            complaintFinal = complaint2
        if row.get("city")=="US_CHICAGO": # if Chicago, concatenate the two
            complaintFinal = row.get("complaint_1") + " "+ complaintFinal
        complaintProcessed = preProcess(complaintFinal)
        if complaintProcessed == "" or re.search('[a-zA-Z]', complaintProcessed) is None: # if nothing or no letters
            return preProcess(row.get("complaint_1"))
        return complaintProcessed
    complaintProcessed = preProcess(row.get("complaint_1"))
    return complaintProcessed

results = dat.apply(lambda row: getComplaint(row),axis=1)
print(results[results.isna()].shape)

(0,)


In [77]:
results[:5]
dat['complaint_1'][:5]

0    questionable business practices regarding oils...
1    constructiondemolition debris dumped in back y...
2    replacing windows and sandblasting causing dus...
4    property causing hazardous conditions on publi...
5    construction work is causing dust  this invest...
dtype: object

0    QUESTIONABLE BUSINESS PRACTICES REGARDING OILS...
1    CONSTRUCTION/DEMOLITION DEBRIS DUMPED IN BACK ...
2    REPLACING WINDOWS AND SANDBLASTING CAUSING DUS...
4    PROPERTY CAUSING HAZARDOUS CONDITIONS ON PUBLI...
5                    CONSTRUCTION WORK IS CAUSING DUST
Name: complaint_1, dtype: object

In [78]:
dat['complaint'] = results
dat.loc[dat['complaint'].isna()].shape

(0, 11)

In [79]:
# Strip white spaces from CATEGORY_SUB
dat['category_sub'] = dat['category_sub'].str.strip()

In [85]:
dat.columns

Index(['index', 'complaint_id', 'city', 'complaint_date', 'dept_311',
       'code_311', 'category_main', 'category_sub', 'complaint_1',
       'complaint_2', 'complaint'],
      dtype='object')

# EDA

Look up top 10 values in columns that aren't ID numbers or dates.

In [127]:
most_common_vals = {}

for col in [x for x in dat.columns if x not in ['index', 'complaint_id', 'complaint_date']]:
    most_common_vals[col] = {}
    most_common_vals[col]['percent'] = dat[col].value_counts(normalize=True)
    most_common_vals[col]['count'] = dat[col].value_counts()

In [133]:
for key, val in most_common_vals.items():
    top_ten = val['count'][:10]
    print(top_ten)

US_CINCINNATI             518774
US_BATON_ROUGE            227137
City of Chicago           124521
US_BLOOMINGTON             82269
City of Oakland - OLD      49767
US_CHICAGO                 48136
IRELAND                    23089
City of Oakland            15629
City of St. Petersburg     14840
New Haven                  13107
Name: city, dtype: int64
public services          325608
garbage                  120177
cinc building dept        72090
cinc health dept          66125
health_environment        48136
recycling                 42583
trash                     36896
city manager's office     24095
general                   23089
blighted properties       21327
Name: dept_311, dtype: int64
metal furniture, spec collectn    159016
general                            82269
tall grass/weeds, private prop     34709
trash, request for collection      30881
pothole, repair                    30320
building, residential              26511
pothole                            24874
missed wo

In [136]:
# Output table of top 10 per col, along with bar chart
for key, val in most_common_vals.items():
    top_ten_count = val['count'][:10]
    top_ten_percent = val['percent'][:10]
    
    # Table
    trace0 = go.Table(
      columnorder = [1,2,3],
      columnwidth = [400,80, 80],
      header = dict(
        values = [[str(key) + '<b>: Value'],
                ['<b>Count</b>'],
                 ['<b>Percent</b>']],
        line = dict(color = '#506784'),
        fill = dict(color = '#4B88A2'),
        align = ['left','center'],
        font = dict(color = 'white', size = 12),
        height = 40
      ),
      cells = dict(
        values = [list(top_ten_count.index), list(top_ten_count.values), list(top_ten_percent.values.round(3))],
        line = dict(color = '#506784'),
        fill = dict(color = ['#D3D4D9', 'white']),
        align = ['left', 'center'],
        font = dict(color = '#506784', size = 12),
        height = 30
    ))

    data = [trace0]

    iplot(data, filename = "Row and Column Size")
    
    # Bar chart
    data = [go.Bar(
        x=list(top_ten.values)[::-1],
        y=list(top_ten.index)[::-1],
        orientation = 'h',
        
    )]

    layout = go.Layout(
        margin=go.layout.Margin(
            l=150,
#             r=0,
#             b=10,
#             t=10,
            pad=4
        ),
        barmode='stack',
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, filename='basic-bar')

### City
Cincinnati makes up 40% of our trouble ticket data. City tickets

In [66]:
# Filter data only to ones with at least 10 characters
mask = (dat['complaint'].str.len() >=10)
dat = dat.loc[mask]
print(dat.shape)

(1084456, 11)


In [8]:
test['city'].value_counts()

US_CINCINNATI                                         518774
US_BATON_ROUGE                                        227137
City of Chicago                                       124521
US_BLOOMINGTON                                         82269
City of Oakland - OLD                                  49768
US_CHICAGO                                             48579
IRELAND                                                23089
City of Oakland                                        15629
City of St. Petersburg                                 14840
New Haven                                              13107
City of Houston                                        12464
DC 311                                                  6690
City of Detroit                                         4920
Neighborhoods                                           4747
Macon-Bibb County                                       4668
City of St. Paul                                        4101
City of Memphis         